In [1]:
import os
import re
import pandas as pd

carpetas=os.walk('.') #quiero resivar todos los archivos y carpetas

#for val in carpetas:
#    print (val)


date_file_dict={} #diccionario vacío para poder tener dentro los archivos
all_files=[] #estarán todos los archivos

for root, dirs, files in os.walk(".", topdown=False): #reconoce todos los archivos de la carpeta en que estoy que terminan con xls
    for name in files:
        if name.endswith("xls"):
            all_files.append(os.path.join(root,name))

for filename in all_files: #reconoce la fecha a partir del nombre del archivo
    g=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", filename)
    date_file_dict.update({g.group(1):filename})

all_df=[]
for v,k in date_file_dict.items(): #v es la fecha y k es la ruta del archivo
    demanda_activa=pd.read_excel(k, 'Erd', skiprows=6)
    demanda_activa=demanda_activa.drop(['Agente'], axis=1)
    df=pd.DataFrame(data=demanda_activa)
    demanda=df.groupby("Barra").sum()
    demanda1=demanda.filter(demanda.columns[1:-1])
    my_date_range=pd.date_range(start=v+ ' 00:00:00', end=v+ ' 23:00:00',freq = 'H')
    #print (my_date_range)
    demanda1=demanda1.rename(columns={'Hora 1':my_date_range[0],'Hora 2':my_date_range[1],'Hora 3':my_date_range[2], 'Hora 4':my_date_range[3],
                      'Hora 5':my_date_range[4],'Hora 6':my_date_range[5], 'Hora 7':my_date_range[6], 'Hora 8':my_date_range[7],
                      'Hora 9':my_date_range[8],'Hora 10':my_date_range[9], 'Hora 11':my_date_range[10], 'Hora 12':my_date_range[11],
                      'Hora 13':my_date_range[12],'Hora 14':my_date_range[13],'Hora 15':my_date_range[14],'Hora 16':my_date_range[15],
                      'Hora 17':my_date_range[16],'Hora 18':my_date_range[17],'Hora 19':my_date_range[18],'Hora 20':my_date_range[19],
                      'Hora 21':my_date_range[20],'Hora 22':my_date_range[21],'Hora 23':my_date_range[22],'Hora 24':my_date_range[23]})
    demanda1['Barra']=demanda1.index
    df1=pd.melt(demanda1,id_vars=['Barra'],value_vars=demanda1.columns[:-1])
    df2=df1.sort_values(['Barra','variable'])
    #df2.reset_index(drop=True)
    all_df.append(df2)

df_final=pd.concat(all_df)
df_final.reset_index(drop=True)


,Barra,variable,value
0,AGO138,2019-11-01 00:00:00,0.000
1,AGO138,2019-11-01 01:00:00,0.000
2,AGO138,2019-11-01 02:00:00,0.000
3,AGO138,2019-11-01 03:00:00,0.000
4,AGO138,2019-11-01 04:00:00,0.000
...,...,...,...
65515,YAN069,2019-11-06 19:00:00,8912.024
65516,YAN069,2019-11-06 20:00:00,8305.669
65517,YAN069,2019-11-06 21:00:00,7544.385
65518,YAN069,2019-11-06 22:00:00,6557.586


In [3]:
df_final.to_csv('demanda1.csv',index=False)